Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AMZN,Amazon,Retailing
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [7]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AMZN,Amazon,US Stocks:Consumer Discretionary
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology


In [8]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AXP AMGN AMZN AAPL BA CAT CVX CSCO KO DIS GS HD HON IBM JNJ JPM MCD MRK MSFT NKE NVDA PG CRM SHW TRV UNH VZ V WMT 


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AMZN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,NVDA,PG,CRM,SHW,TRV,UNH,VZ,V,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,125.57,87.83,158.59,76.96,37.67,314.65,109.72,84.23,35.50,38.96,...,69.19,3.38,78.00,134.80,123.90,102.64,222.48,40.08,127.27,28.29
2019-01-03,120.84,86.11,156.17,75.01,33.92,302.10,105.49,82.62,34.21,38.72,...,67.96,3.17,77.45,129.68,120.16,100.79,216.41,40.22,122.69,28.15
2019-01-04,125.81,89.99,161.51,78.77,35.36,317.82,111.26,84.33,35.75,39.49,...,69.74,3.38,79.03,137.19,123.35,103.78,218.94,40.32,127.97,28.32
2019-01-07,125.52,90.48,163.69,81.48,35.28,318.82,111.32,85.43,35.99,38.97,...,70.74,3.56,78.72,141.43,125.09,103.45,219.36,40.58,130.28,28.66
2019-01-08,126.05,90.92,165.78,82.83,35.96,330.89,112.66,85.05,36.28,39.41,...,71.68,3.47,79.01,144.91,125.97,103.05,222.29,41.77,130.99,28.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,145.48,295.53,309.72,212.71,247.10,179.91,338.92,157.89,63.24,70.59,...,80.28,130.28,170.54,308.32,343.87,248.24,461.52,43.32,349.86,93.67
2025-02-25,146.54,293.34,315.63,212.80,247.04,178.27,342.36,156.42,64.27,71.49,...,81.72,126.63,172.65,305.88,353.03,252.51,470.38,43.71,352.09,97.69
2025-02-26,147.43,296.10,306.38,214.35,240.36,173.04,342.58,155.14,64.08,70.80,...,81.17,131.28,171.30,307.33,359.59,248.85,463.59,43.18,350.63,96.20


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(105.57895333932655)

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.20245577221438657)

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.1194472788933945)

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()
bb.print_full(monthly[symbol])

Date
2019-01-31   131.72
2019-02-28   137.32
2019-03-31   137.58
2019-04-30   125.48
2019-05-31   106.68
2019-06-30   115.76
2019-07-31   116.68
2019-08-31   108.99
2019-09-30   110.79
2019-10-31   111.19
2019-11-30   115.40
2019-12-31   119.92
2020-01-31   107.85
2020-02-29   102.37
2020-03-31    93.63
2020-04-30   104.20
2020-05-31   108.37
2020-06-30   108.06
2020-07-31   104.23
2020-08-31   113.96
2020-09-30   111.97
2020-10-31   111.82
2020-11-30   121.79
2020-12-31   123.24
2021-01-31   123.85
2021-02-28   124.45
2021-03-31   136.97
2021-04-30   140.14
2021-05-31   145.40
2021-06-30   142.24
2021-07-31   141.75
2021-08-31   140.52
2021-09-30   126.58
2021-10-31   128.93
2021-11-30   123.70
2021-12-31   129.22
2022-01-31   120.77
2022-02-28   109.18
2022-03-31   109.35
2022-04-30   105.93
2022-05-31   110.76
2022-06-30    96.01
2022-07-31   106.27
2022-08-31    93.20
2022-09-30    82.82
2022-10-31    94.28
2022-11-30    95.52
2022-12-31    90.94
2023-01-31    87.27
2023-02-28    8

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2019-01-31     NaN
2019-02-28    0.04
2019-03-31    0.00
2019-04-30   -0.09
2019-05-31   -0.15
              ... 
2024-10-31   -0.06
2024-11-30    0.04
2024-12-31   -0.03
2025-01-31    0.18
2025-02-28    0.02
Freq: ME, Name: MMM, Length: 74, dtype: float64

In [18]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

np.float64(0.318815027427374)

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')